In [24]:
import pandas as pd
import re

# Read the list of filenames from the configuration file
with open('../file_list.txt', 'r', encoding='utf-8') as config_file:
    file_names = config_file.read().splitlines()

# Regex pattern to match the data format
pattern = r'\[(.*?)\] (.*?): (.*)'

# Initialize an empty list to store parsed data
datalist = []
stream_count = 0
# Iterate over each specified file
for file in file_names:
    full_path = f"../data/{file}"
    with open(full_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        for line in lines:
            match = re.match(pattern, line)
            if match:
                date, user, message = match.groups()
                datalist.append([date, user, message,stream_count])
    stream_count = stream_count + 1

# Create a DataFrame from the parsed data
data = pd.DataFrame(datalist, columns=["date", "user", "message","stream"])


In [25]:
data = data[data["user"] != "StreamElements"]
data = data[data["user"] != "Fossabot"]

In [26]:

data['date'] = pd.to_datetime(data['date'])

data = data[data['date'].dt.year == 2026]



In [27]:
data["user"] = data["user"].replace("Banties1g", "banties_x")
data["user"] = data["user"].replace("banties1g", "banties_x")
data["user"] = data["user"].replace("chili_poe", "chili_con_bacon")
data["user"] = data["user"].replace("CHILI_POE", "chili_con_bacon")
data["user"] = data["user"].replace("Chili_poe", "chili_con_bacon")
data["user"] = data["user"].replace("chili_conbacon", "chili_con_bacon")
data["user"] = data["user"].replace("Wirelesss_", "W1r3lesss")
data["user"] = data["user"].replace("treklul", "trek44_")
data["user"] = data["user"].replace("ttrek_", "trek44_")
data["user"] = data["user"].replace("trek_x", "trek44_")
data["user"] = data["user"].replace("TriplesingleJ", "TripleSingleJames")
data["user"] = data["user"].replace("uwu_cougar", "uuccugr")
data["user"] = data["user"].replace("uuccugr_","uuccugr")
data["user"] = data["user"].replace("StanIV4_", "stan_iv4")
data["user"] = data["user"].replace("Muuskie2", "Muuskie")
data["user"] = data["user"].replace("nishad_more1311", "nishad13")
data["user"] = data["user"].replace("softarballt", "softarr")
data["user"] = data["user"].replace("softarballtt23", "softarr")
data["user"] = data["user"].replace("lajosbarnabas", "lajoss__")
data["user"] = data["user"].replace("Bonkwiththefunk", "bonk67")
data["user"] = data["user"].replace("qfishyy11", "bonk67")

In [28]:
from collections import defaultdict

# Get all unique usernames
unique_users = data['user'].unique()

# Create a mapping from lowercase username to all variants

user_variants = defaultdict(set)
for user in unique_users:
    user_variants[user.lower()].add(user)

# Find usernames with different capitalization
duplicate_users = {k: v for k, v in user_variants.items() if len(v) > 1}

In [29]:
# Create a mapping from all variants to the canonical (sorted first) variant
variant_map = {}
for variants in duplicate_users.values():
    sorted_variants = sorted(variants)
    canonical = sorted_variants[0]
    for v in variants:
        variant_map[v] = canonical

# Replace usernames in 'user' column
data['user'] = data['user'].apply(lambda u: variant_map.get(u, u))

In [30]:
# Count the number of messages per user
message_counts = data.groupby("user")["message"].count()

# Filter users with 25 or more messages
users_with_25_or_more = message_counts[message_counts >= 25].index

# Filter the original DataFrame to keep only these users
data = data[data["user"].isin(users_with_25_or_more)]

In [31]:
from collections import defaultdict

# Get all unique usernames
unique_users = data['user'].unique()

# Create a mapping from lowercase username to all variants

user_variants = defaultdict(set)
for user in unique_users:
    user_variants[user.lower()].add(user)

# Find usernames with different capitalization
duplicate_users = {k: v for k, v in user_variants.items() if len(v) > 1}

In [32]:
# Create a mapping from all variants to the canonical (sorted first) variant
variant_map = {}
for variants in duplicate_users.values():
    sorted_variants = sorted(variants)
    canonical = sorted_variants[0]
    for v in variants:
        variant_map[v] = canonical

# Replace usernames in 'user' column
data['user'] = data['user'].apply(lambda u: variant_map.get(u, u))

In [33]:
# Convert date to datetime format
data["date"] = pd.to_datetime(data["date"])



In [34]:
# Truncate datetime to just the day (removing time)
data["day"] = data["date"].dt.date  # Extract only the date part



In [35]:
# Group by 'day' and 'user' and calculate the message count per day per user
data["message_count"] = 1  # Assign 1 for each message to count them
daily_counts = data.groupby(["day", "user"])["message_count"].count().reset_index()



In [36]:
# Pivot the table to create a user-wise table for each day
pivot_table = daily_counts.pivot(index="day", columns="user", values="message_count").fillna(0)


In [37]:

# Add a cumulative sum for each user across the days
cumulative_pivot = pivot_table.cumsum()


In [38]:

# Print the result
print(cumulative_pivot)

user        07moki  0LcK_  0buddha  0ooma  10_DiE  1206paul_  16GP4  176kuba  \
day                                                                            
2026-01-01     0.0    0.0      0.0    0.0     0.0        0.0    0.0      0.0   
2026-01-03     0.0    0.0     23.0    0.0     0.0        0.0    0.0      0.0   
2026-01-04     0.0    0.0     23.0    2.0     0.0        7.0    0.0      0.0   
2026-01-05     0.0    0.0     32.0    3.0     0.0       31.0    0.0      2.0   
2026-01-06     0.0    0.0     45.0    3.0     0.0       55.0    0.0      2.0   
2026-01-09     0.0    7.0     52.0    3.0     0.0       55.0    0.0      2.0   
2026-01-10    19.0   30.0     56.0    3.0     0.0       56.0    2.0      3.0   
2026-01-12    19.0   30.0     60.0    3.0     0.0       56.0    5.0      4.0   
2026-01-13    19.0   30.0     60.0    3.0     0.0       65.0    5.0      7.0   
2026-01-14    19.0   30.0     77.0    3.0     0.0       66.0    5.0      7.0   
2026-01-15    19.0   30.0     77.0    3.

In [39]:
pivot_data_cleaned_transposed = cumulative_pivot.T
pivot_data_cleaned_transposed = cumulative_pivot.T.reset_index()

In [40]:
pivot_data_cleaned_transposed.head(5)
pivot_data_cleaned_transposed.tail(5)

day,user,2026-01-01,2026-01-03,2026-01-04,2026-01-05,2026-01-06,2026-01-09,2026-01-10,2026-01-12,2026-01-13,...,2026-02-07,2026-02-08,2026-02-09,2026-02-10,2026-02-11,2026-02-12,2026-02-13,2026-02-14,2026-02-15,2026-02-16
1044,zxkre,0.0,1.0,1.0,9.0,16.0,16.0,33.0,33.0,34.0,...,54.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0,55.0
1045,محمدحسين,0.0,39.0,39.0,39.0,39.0,39.0,39.0,39.0,39.0,...,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0,40.0
1046,金龙,0.0,0.0,0.0,1.0,1.0,11.0,20.0,20.0,21.0,...,27.0,27.0,27.0,28.0,28.0,28.0,31.0,31.0,31.0,31.0
1047,빵댕이ㅋ,2.0,7.0,7.0,9.0,10.0,12.0,15.0,17.0,18.0,...,37.0,37.0,37.0,37.0,37.0,37.0,38.0,38.0,38.0,38.0
1048,알래스카해달,95.0,95.0,95.0,193.0,193.0,272.0,274.0,274.0,302.0,...,1041.0,1151.0,1164.0,1191.0,1202.0,1202.0,1202.0,1230.0,1282.0,1313.0


In [41]:
# --- Step 1 & 2: Calculate rank and identify top 20 users ---

# Calculate rank for each date (column) in descending order.
# The user with the highest cumulative sum gets rank 1.
ranked_df = pivot_data_cleaned_transposed.rank(axis=0, ascending=False, method='min')

# Identify users who were in the top 20 on at least one date.
# This creates a boolean Series where True means the user was in the top 20 at least once.
users_in_top_15_at_least_once = (ranked_df <= 20).any(axis=1)

# Get the list of users (their index labels) who meet the criteria.
users_to_keep = users_in_top_15_at_least_once[users_in_top_15_at_least_once].index

# --- Step 3: Filter the DataFrame ---

# Create a new DataFrame containing only the users who were in the top 20 at least once.
filtered_users_df = pivot_data_cleaned_transposed.loc[users_to_keep]

print("Original DataFrame shape:", pivot_data_cleaned_transposed.shape)
print("Filtered DataFrame shape:", filtered_users_df.shape)

Original DataFrame shape: (1049, 38)
Filtered DataFrame shape: (55, 38)


In [42]:
filtered_users_df.head()

day,user,2026-01-01,2026-01-03,2026-01-04,2026-01-05,2026-01-06,2026-01-09,2026-01-10,2026-01-12,2026-01-13,...,2026-02-07,2026-02-08,2026-02-09,2026-02-10,2026-02-11,2026-02-12,2026-02-13,2026-02-14,2026-02-15,2026-02-16
34,Aluminiumminimumimmunity,41.0,116.0,169.0,258.0,354.0,384.0,480.0,558.0,592.0,...,1876.0,1876.0,1876.0,1893.0,1899.0,1899.0,1930.0,1932.0,1932.0,1967.0
48,Balicious1,43.0,109.0,117.0,127.0,128.0,128.0,128.0,128.0,128.0,...,149.0,149.0,149.0,149.0,149.0,151.0,151.0,152.0,152.0,153.0
50,BenXBari,17.0,17.0,17.0,20.0,46.0,60.0,335.0,494.0,494.0,...,2358.0,2534.0,2722.0,2735.0,2762.0,3023.0,3440.0,3662.0,3948.0,4220.0
76,CrystalMethod1000,43.0,99.0,106.0,127.0,206.0,272.0,367.0,478.0,588.0,...,2434.0,2544.0,2612.0,2682.0,2846.0,2848.0,2973.0,3004.0,3026.0,3138.0
113,FilipStayout,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1642.0,1907.0,2195.0,3078.0,3412.0,3754.0,4321.0,4680.0,5107.0,5870.0


In [43]:
filtered_users_df.info

<bound method DataFrame.info of day                       user  2026-01-01  2026-01-03  2026-01-04  \
34    Aluminiumminimumimmunity        41.0       116.0       169.0   
48                  Balicious1        43.0       109.0       117.0   
50                    BenXBari        17.0        17.0        17.0   
76           CrystalMethod1000        43.0        99.0       106.0   
113               FilipStayout         0.0         0.0         0.0   
114                  Foolsworn         9.0         9.0         9.0   
141                   HALP____        56.0       297.0       491.0   
157               Iiidxnm12iii        34.0       106.0       136.0   
159            InverseEntropy_         6.0         7.0        63.0   
165                   Ivana_10        40.0       109.0       155.0   
166                   JBIN2036       224.0       601.0       840.0   
176                   KRIESEAX         7.0       223.0       315.0   
229                    Muuskie        52.0       316.0    

In [44]:
filtered_users_df.head()

day,user,2026-01-01,2026-01-03,2026-01-04,2026-01-05,2026-01-06,2026-01-09,2026-01-10,2026-01-12,2026-01-13,...,2026-02-07,2026-02-08,2026-02-09,2026-02-10,2026-02-11,2026-02-12,2026-02-13,2026-02-14,2026-02-15,2026-02-16
34,Aluminiumminimumimmunity,41.0,116.0,169.0,258.0,354.0,384.0,480.0,558.0,592.0,...,1876.0,1876.0,1876.0,1893.0,1899.0,1899.0,1930.0,1932.0,1932.0,1967.0
48,Balicious1,43.0,109.0,117.0,127.0,128.0,128.0,128.0,128.0,128.0,...,149.0,149.0,149.0,149.0,149.0,151.0,151.0,152.0,152.0,153.0
50,BenXBari,17.0,17.0,17.0,20.0,46.0,60.0,335.0,494.0,494.0,...,2358.0,2534.0,2722.0,2735.0,2762.0,3023.0,3440.0,3662.0,3948.0,4220.0
76,CrystalMethod1000,43.0,99.0,106.0,127.0,206.0,272.0,367.0,478.0,588.0,...,2434.0,2544.0,2612.0,2682.0,2846.0,2848.0,2973.0,3004.0,3026.0,3138.0
113,FilipStayout,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1642.0,1907.0,2195.0,3078.0,3412.0,3754.0,4321.0,4680.0,5107.0,5870.0


In [45]:
filtered_users_df.to_excel('chattersRace2026.xlsx', sheet_name='Pivot Table')

In [46]:
data.shape

(301227, 6)